# 8. Building Complex Models with the Functional API

## 8.1. Introductie tot de Functional API

Waar het `Sequential` model een simpele stapel lagen is, kun je met de **Functional API** complexere architecturen bouwen die niet-sequentieel zijn. Een bekend voorbeeld is het **Wide & Deep** netwerk (Cheng et al., 2016).

## 8.2. Het Wide & Deep Netwerk (Basis)

In dit model wordt de input direct verbonden met de output (het **Wide** gedeelte) én via een reeks verborgen lagen (het **Deep** gedeelte).

### Constructie via symbolische verbindingen

In de Functional API roep je lagen aan als functies om verbindingen te leggen. Er wordt op dit punt nog geen echte data verwerkt.

In [ ]:

import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# 1. Laad de juiste data (numerieke tabeldata in plaats van afbeeldingen)
housing = fetch_california_housing()

# 2. Splitsen
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

1. Lagen definiëren

    Eerst definiëren we de "bouwstenen" van het netwerk. Door ze hier als variabelen op te slaan, kunnen we ze later specifiek aanroepen (bijvoorbeeld om de `normalization_layer` te trainen op de data statistieken).

In [52]:
normalization_layer = keras.layers.Normalization()
hidden_layer1 = keras.layers.Dense(30, activation="relu")
hidden_layer2 = keras.layers.Dense(30, activation="relu")
concat_layer = keras.layers.Concatenate()
output_layer = keras.layers.Dense(1)

2. Verbindingen leggen (symbolisch)

    Hier "bedraden" we het netwerk. Elke laag wordt aangeroepen met de vorige laag (of een lijst van lagen) als argument. Merk op dat de concat_layer zowel de directe input als de diepe output ontvangt, wat het een Wide & Deep architectuur maakt.

In [53]:
input_ = keras.layers.Input(shape=X_train.shape[1:]) # Input vorm specificeren
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2]) # Concatenate voegt inputs samen
output = output_layer(concat)

3. Het Model object creëren

    Ten slotte bundelen we alles in een Model object. Omdat alle lagen symbolisch aan elkaar gelinkt zijn, hoef je Keras alleen te vertellen waar de stroom begint (inputs) en waar deze eindigt (outputs).

In [54]:
model = keras.Model(inputs=[input_], outputs=[output])
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_20    │ (None, 8)         │         17 │ input_layer_5[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 30)        │        270 │ normalization_20… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 30)        │        930 │ dense_32[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, 38)        │          0 │ normalization_20… │
│ (Concatenate)       │                   │            │ dense_33[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 1)         │         39 │ concatenate_13[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,256 (4.91 KB)

 Trainable params: 1,239 (4.84 KB)

 Non-trainable params: 17 (72.00 B)

## 3. Multiple Inputs & Multiple Outputs

Soms wil je verschillende subsets van features naar verschillende paden sturen, of heb je meerdere outputs nodig (bijv. voor classificatie én lokalisatie).

### Architectuur opzet

- **Wide pad:** Neemt bijvoorbeeld de eerste 5 features (0 t/m 4).
- **Deep pad:** Neemt de laatste 6 features (2 t/m 7).
- **Auxiliary Output:** Een extra output bovenop de verborgen lagen, vaak gebruikt voor **regularisatie**.

### Implementatie voorbeeld

In [55]:
input_wide = keras.layers.Input(shape=[5], name="wide_input")
input_deep = keras.layers.Input(shape=[6], name="deep_input")

norm_wide = keras.layers.Normalization()(input_wide)
norm_deep = keras.layers.Normalization()(input_deep)

hidden1 = keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

concat = keras.layers.concatenate([norm_wide, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

model = keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

## Multiple Inputs and Multiple Outputs

In complexe scenario's volstaat een enkel pad niet. We bouwen hier een architectuur met **twee inputs** (Wide & Deep) en **twee outputs** (Main & Auxiliary).

**Waarom meerdere inputs/outputs?**

- **Meerdere taken:** Bijvoorbeeld tegelijkertijd een object classificeren én de locatie (bounding box) bepalen in een afbeelding.
- **Regularisatie:** Een extra output (auxiliary) toevoegen aan een deel van het netwerk om te zorgen dat ook die lagen nuttige representaties leren.
- **Verschillende databronnen:** Verschillende subsets van features naar verschillende delen van het model sturen.

**1. Model Architectuur opbouwen**

In dit voorbeeld sturen we 5 features naar het 'Wide' pad en 6 features naar het 'Deep' pad (met overlap).

In [56]:
import keras

# Inputs definiëren
input_wide = keras.layers.Input(shape=[5], name="wide_input")  # Features 0 t/m 4
input_deep = keras.layers.Input(shape=[6], name="deep_input")  # Features 2 t/m 7

# Normalisatie lagen (apart opgeslagen voor .adapt())
norm_layer_wide = keras.layers.Normalization()
norm_layer_deep = keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Deep pad: we maken en roepen Dense lagen in één regel aan voor beknoptheid
hidden1 = keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

# Paden samenvoegen
# Tip: keras.layers.concatenate (kleine letter) is een shortcut functie
concat = keras.layers.concatenate([norm_wide, hidden2])

# Outputs definiëren
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

# Het Model object
model = keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

**2. Compileren met Loss Weights**

Omdat we twee outputs hebben, moeten we twee losses opgeven. We geven de hoofdoutput meer gewicht ($0.9$) dan de hulpoutput ($0.1$).

In [57]:
# Data splitsen voor de twee paden
X_train_wide,X_train_deep=X_train[:,:5],X_train[:,2:]
X_valid_wide,X_valid_deep=X_valid[:,:5],X_valid[:,2:]
X_test_wide,X_test_deep=X_test[:,:5],X_test[:,2:]

# pretend new instances
X_new_wide,X_new_deep=X_test_wide[:3],X_test_deep[:3]


optimizer = keras.optimizers.Adam(learning_rate=1e-3)

model.compile(
    loss=("mse", "mse"),
    loss_weights=(0.9, 0.1),
    optimizer=optimizer,
   metrics={"main_output": ["RootMeanSquaredError"], "aux_output": ["RootMeanSquaredError"]}
)

**3. Data Splitsen en Training**

We moeten de data handmatig splitsen zodat de juiste kolommen naar de juiste input-laag gaan.

In [58]:

# Normalisatie lagen trainen
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

# Trainen: geef inputs en targets door als tuples
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train),
    epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 735us/step - aux_output_RootMeanSquaredError: 1.4619 - aux_output_loss: 2.1367 - loss: 1.6332 - main_output_RootMeanSquaredError: 1.2559 - main_output_loss: 1.5770 - val_aux_output_RootMeanSquaredError: 2.4736 - val_aux_output_loss: 6.1157 - val_loss: 1.6771 - val_main_output_RootMeanSquaredError: 1.0880 - val_main_output_loss: 1.1832
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - aux_output_RootMeanSquaredError: 0.8510 - aux_output_loss: 0.7241 - loss: 0.5706 - main_output_RootMeanSquaredError: 0.7440 - main_output_loss: 0.5535 - val_aux_output_RootMeanSquaredError: 1.7535 - val_aux_output_loss: 3.0733 - val_loss: 0.7529 - val_main_output_RootMeanSquaredError: 0.7035 - val_main_output_loss: 0.4948
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - aux_output_RootMeanSquaredError: 0.7730 - aux_output_loss: 0.5977 - loss: 0.4856 - main_output_RootMeanSquaredError: 0.6879 - main_output_loss: 0.4735 - val_aux_output_RootMeanSquaredE

**4. Evaluatie en Voorspelling**

De resultaten van `evaluate` en `predict` worden nu teruggegeven als lijsten of tuples.

Evaluatie

In [59]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
# Geeft: [weighted_sum_loss, main_loss, aux_loss, main_rmse, aux_rmse]
weighted_sum, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step - aux_output_RootMeanSquaredError: 0.6199 - aux_output_loss: 0.3843 - loss: 0.3265 - main_output_RootMeanSquaredError: 0.5658 - main_output_loss: 0.3196


Voorspellen

In [ ]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred_main, y_pred_aux = y_pred_tuple

# Omzetten naar een dictionary voor duidelijkheid:
y_pred_dict = dict(zip(model.output_names, y_pred_tuple))
y_pred_dict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


{'main_output': array([[0.5522039],
        [1.1439954],
        [3.7478845]], dtype=float32),
 'aux_output': array([[0.73428047],
        [1.0183663 ],
        [3.560885  ]], dtype=float32)}